In [3]:
from sys import path
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt

In [4]:
path.append("/Users/rachaelmcalister/warehouse-script/python/ab_weekly_tests/")
path.append("/Users/rachaelmcalister/jupyter_repo/")

In [9]:
from db_connector_user import DBConnectorUser
from db_connection_user import DBConnectionUser
import pandas as pd
from useful_functions import executeScriptsFromFile, calculate_conversion_CIs

In [10]:
from yaml_util import *

# Setup Database Connection

In [11]:
db_settings_dict = load_yaml_file('/Users/rachaelmcalister/jupyter_repo/db_settings.yml')
db_settings = db_settings_dict.get('db_settings')
DBConnectorUser(db_settings)
DBConnectionUser.setup_db_settings(db_settings)

# Import Data

In [12]:
df = executeScriptsFromFile('sql_queries/predict_same_size_as_signup_size', DBConnectionUser)

# Results

In [13]:
df['quantity_correct'] = df.quantity*df.correct_pred
df['return_quantity_correct'] = df.return_quantity*df.correct_pred
df['kept_quantity_correct'] = df.quantity_correct - df.return_quantity_correct

In [14]:
garments = [1, 6, 9, 12]
df = df[df.garment_category_id.isin(garments)]

In [15]:
df_agg = pd.DataFrame(df.groupby(['garment_category_id']).agg({'garment_category_id':'size', 'quantity': 'sum',
                                         'return_quantity': 'sum', 'correct_pred': 'sum',
                                        'quantity_correct': 'sum', 'return_quantity_correct': 'sum',
                                                              'kept_quantity_correct': 'sum'}).rename(columns={
    'garment_category_id': 'num_entries',
    'correct_pred': 'num_entries_correct',
})).reset_index()

In [16]:
# Include baseline % kept for comparison
df_agg['pct_quantity_correct_kept'] = (df_agg.kept_quantity_correct)/df_agg.quantity_correct
df_agg['pct_quantity_kept'] = (df_agg.quantity-df_agg.return_quantity)/df_agg.quantity


In [19]:
# Calculate 95% confidence intervals
df_agg['quantity_correct_kept_CI'] = df_agg.apply(lambda x: calculate_conversion_CIs(x.pct_quantity_correct_kept, x.quantity_correct), axis=1)
df_agg['quantity_kept_CI'] = df_agg.apply(lambda x: calculate_conversion_CIs(x.pct_quantity_kept, x.quantity), axis=1)
df_agg.set_index('garment_category_id', inplace=True)


In [20]:
df_agg[df_agg.columns[:7]]


,num_entries,quantity,return_quantity,num_entries_correct,quantity_correct,return_quantity_correct,kept_quantity_correct
garment_category_id,,,,,,,
1,2189,2191,1396,1235.0,1236,774,462
6,1904,1921,862,1045.0,1052,485,567
9,502,505,267,285.0,286,145,141
12,1288,1290,584,603.0,604,267,337


In [21]:
df_agg[df_agg.columns[7:]]

,pct_quantity_correct_kept,pct_quantity_kept,quantity_correct_kept_CI,quantity_kept_CI
garment_category_id,,,,
1,0.373786,0.362848,"[0.347, 0.401]","[0.343, 0.383]"
6,0.538973,0.551275,"[0.509, 0.569]","[0.529, 0.574]"
9,0.493007,0.471287,"[0.435, 0.551]","[0.428, 0.515]"
12,0.557947,0.547287,"[0.518, 0.598]","[0.52, 0.574]"
